In [1]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json


load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]

def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    if result.status_code ==200:
        json_result=json.loads(result.content)
        return json_result['tracks']
    else:
        print(f"Failed to fetch recommendations. Status code: {result.status_code}")
        return None


In [3]:
def search_for_song(token, song_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={song_name}&type=track&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    if result.status_code == 200:
        json_result = json.loads(result.content)
        return json_result
    else:   
        print(f"Failed to find song. Status code: {result.status_code}")
        return None
    
    
    

In [4]:
token = get_token()


In [5]:
song_name = input("What song do you want?")

In [6]:
answer = search_for_song(token, song_name)

In [9]:

answer['tracks']['items'][0]['id']

'5TRPicyLGbAF2LGBFbHGvO'

In [12]:

answer['tracks']['items'][0]['album']['images'][1]['url']

'https://i.scdn.co/image/ab67616d00001e0226f7f19c7f0381e56156c94a'